# **Task I: Quantum Computing Part**





###1) implement a simple quantum operation with Cirq or Pennylane

a) With 5 qubits 

b) Apply Hadamard operation on every qubit

c) Apply CNOT operation on (0, 1), (1,2), (2,3), (3,4) 

d) SWAP (0, 4) 

e) Rotate X with pi/2 on any qubit 

f) Plot the circuit 


In [1]:
!pip install pennylane

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 48.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 KB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.3 MB/s eta 0:00:00


In [2]:
import pennylane as qml
from pennylane import numpy as np

In [3]:
dev=qml.device("default.qubit",wires=5)

@qml.qnode(dev)
def circuit(params):
  qml.Hadamard(wires=0)
  qml.Hadamard(wires=1)
  qml.Hadamard(wires=2)
  qml.Hadamard(wires=3)
  qml.Hadamard(wires=4)

  qml.CNOT(wires=[0,1])
  qml.CNOT(wires=[1,2])
  qml.CNOT(wires=[2,3])
  qml.CNOT(wires=[3,4])

  qml.SWAP(wires=[0,4])

  qml.RX(params[0],wires=2)
  
  return qml.probs(wires=[0, 1, 2, 3, 4])

In [4]:
params=np.array([np.pi/2])
print(qml.draw(circuit)(params))

0: ──H─╭●──────────╭SWAP───────────┤ ╭Probs
1: ──H─╰X─╭●───────│───────────────┤ ├Probs
2: ──H────╰X─╭●────│──────RX(1.57)─┤ ├Probs
3: ──H───────╰X─╭●─│───────────────┤ ├Probs
4: ──H──────────╰X─╰SWAP───────────┤ ╰Probs


###2) Implement a second circuit with a framework of your choice:
a) Apply a Hadmard gate to the first qubit

b) rotate the second qubit by pi/3 around X

c) Apply Hadamard gate to the third and fourth qubit

d) Perform a swap test between the states of the first and second qubit |q1 q2> and the third and fourth qubit |q3 q4>


#### In order to perform swap test between Multi State system we need to perform swap test twice as stated in [Multi-state Swap Test algorithm] paper
[Multi-state Swap Test algorithm]:https://arxiv.org/pdf/2205.07171.pdf

#####The **New Multi-state Swap Test algorithm** paper states that, If the initial states are s1s2 = |+>|+> and q1q2q3q4 = |φ1> |φ2> |φ3> |φ4>. 
#####Two swap rules are designed as follows:N (N=2^2, 2^3,....2^k) , quantum states can be divided into four groups G1, G2, G3, G4. Rule1 is to exchange the states of 2nd and 3rd group in turn and get G1, G3, G2, G4; Rule2 is to exchange the states of 2nd and 4th group in turn and get G1, G4, G3, G2.

#####We divide q1q2q3q4 into four groups G1 = q1; G2 = q2; G3 = q3; G4 = q4. There are also two CSWAP gates, where s1(s2) is the control qubit to swap two groups G2, G3(q2, q4) according to rule1(rule2). Two simple Swap Test for two quantum states are places at the end of U4 to measure the overlap for q1, q2 and q3, q4.


######*For us in the question we have to measure overlap between the states of |q1q2> and |q3q4>,hence in order to obtain overlap of |q1q2> and |q3q4> as per the paper our s1s2 should be |00>.*


In [6]:
dev=qml.device("default.qubit",wires=['s1','s2','q1','q2','q3','q4'])

@qml.qnode(dev)
def circuit_swap_test(params):
  qml.BasisState(params[0],wires=['s1','s2']) #Initializing the basic states of s1s2 as |00>
  qml.Hadamard(wires='q1') #hadamard gate on first qubit
  qml.RX(params[1],wires='q2') #rotation by pi/3 around X on qubit 2
  qml.Hadamard(wires='q3') # hadamard gate on qubits 3
  qml.Hadamard(wires='q4') # hadamard gate on qubits 4

  qml.CSWAP(wires=['s1','q2','q3']) #By rule 1 swapping G2 and G3 with S1 as ancillary qubit
  qml.CSWAP(wires=['s2','q2','q4']) #By rule 2 swapping G2 and G4 with S2 as ancillary qubit 

  return qml.expval(qml.PauliZ(wires='q1'))

In [7]:
parameters=np.array([np.array([0]),np.pi/3])
print(qml.draw(circuit_swap_test)(parameters))

s1: ─╭BasisState(M0)─╭●──────────┤     
s2: ─╰BasisState(M0)─│─────╭●────┤     
q1: ──H──────────────│─────│─────┤  <Z>
q2: ──RX(1.05)───────├SWAP─├SWAP─┤     
q3: ──H──────────────╰SWAP─│─────┤     
q4: ──H────────────────────╰SWAP─┤     


/usr/local/lib/python3.8/dist-packages/autograd/numpy/numpy_wrapper.py:77: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return _np.array(args, *array_args, **array_kwargs)
